<a href="https://colab.research.google.com/github/vlasek/test/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22ChatBot2_ipynb2%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import nltk
import json # загрузка датасета
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # векторайзер
from sklearn.linear_model import LogisticRegression # классификатор
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split # разбиение на тренировочную и тестовую выборки
# импортируем библиотеки

In [2]:
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': ['Приветики', 'Здравствуй!', 'Здарова', 'Привет'],
#             'responses': ['Хай!!', 'Прив', 'Хэллоу']
#         },
#         'bye': {
#             'examples': ['Пока', 'Сайонара', 'Покедова'],
#             'responses': ['До свиданья', 'Увидимся', 'Пока-пока']
#         },
#         'howdoyoudo': {
#             'examples': ['Как дела?', 'Как поживаешь?'],
#             'responses': ['Классно!', 'Я живу в компьютере']
#         },
#     }
# }

In [3]:
def clean(text):
  cleaned_text = ''
  for char in text.lower():
    if char in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
      cleaned_text = cleaned_text + char
  return cleaned_text

In [4]:
with open('/content/BOT_CONFIG.json') as f:
  BOT_CONFIG = json.load(f)
len(BOT_CONFIG['intents']) # загрузка датасета, 147 интентов

147

In [5]:
corpus = []
y = []
for intent in BOT_CONFIG['intents'].keys():
  for example in BOT_CONFIG['intents'][intent]['examples']:
    corpus.append(example)
    y.append(intent)
len(corpus), len(y) # список текстов и список их интентов в том одинаковом порядке

(559, 559)

In [45]:
corpus_train, corpus_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=42) # разбиение на тренировочную и тестовую выборки

In [62]:
vectorizer = CountVectorizer(ngram_range=(2,2), analyzer='char_wb') # TfidfVectorizer() preprocessor=clean # векторизуем тексты (на тренировочной создаем словарь, к тестовой только применяем)
X_train = vectorizer.fit_transform(corpus_train)
X_test = vectorizer.transform(corpus_test)
print(len(vectorizer.get_feature_names_out())) # длина словаря - 425 слов

557


In [63]:
model = RandomForestClassifier(n_estimators=500) # LogisticRegression(max_iter=200)#
model.fit(X_train, y_train) # учим модель на тренировочной части
model.score(X_train, y_train), model.score(X_test, y_test) # оцениваем качество на тренировочной и на тестовой

(0.8747203579418344, 0.25)

In [64]:
with open('/content/vectorizer.pickle', 'wb') as f:
  pickle.dump(vectorizer, f)
with open('/content/classifier.pickle', 'wb') as f:
  pickle.dump(model, f)

In [65]:
with open('/content/classifier.pickle', 'rb') as f:
  model = pickle.load(f)

In [66]:
def clean(text):
  cleaned_text = ''
  for char in text.lower():
    if char in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя':
      cleaned_text = cleaned_text + char
  return cleaned_text

# def get_intent(text):
#   for intent in BOT_CONFIG['intents'].keys():
#     for example in BOT_CONFIG['intents'][intent]['examples']:
#       clean_example = clean(example)
#       clean_text = clean(text)
#       if nltk.edit_distance(clean_example, clean_text) / max(len(clean_example), len(clean_text)) < 0.4:
#         return intent
#   return 'intent not found'

def get_intent_by_model(text):
  return model.predict(vectorizer.transform([text]))[0]

def bot(text):
  # intent = get_intent(text)
  # if intent == 'intent not found':
  #   return 'Я ничего не понял :('
  # else:
  #   return random.choice(BOT_CONFIG['intents'][intent]['responses'])

  intent = get_intent_by_model(text)
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [67]:
while True:
  text = input()
  answer = bot(text)
  print(answer)

чат бот привет
хай
как настроение
отлично
у меня тоже
2
какая погода
облачно
идет дождь
доброе!
привет
хэллоу
off
увидимся


KeyboardInterrupt: ignored